In [16]:
from transformers import AutoTokenizer, AutoConfig, set_seed
import torch
from torch.utils.data import DataLoader
from load_data_for_R import *
from model_for_R import *
from tqdm import tqdm
import numpy as np
import torch.nn.functional as F

def inference(model, tokenized_sent, device):
	dataloader = DataLoader(tokenized_sent, batch_size=32, shuffle=False, collate_fn=collate_fn)
	model.eval()
	output_pred = []
	output_prob = []
	for i, data in enumerate(tqdm(dataloader)):
		with torch.no_grad():
			outputs = model(
			input_ids = data['input_ids'].to(device),
			attention_mask = data['attention_mask'].to(device),
			sub_mask = data['sub_mask'].to(device),
			obj_mask = data['obj_mask'].to(device),
			labels = None
			)

			logits = outputs[0]
			prob = F.softmax(logits, dim=-1).detach().cpu().numpy()
			logits = logits.detach().cpu().numpy()
			result = np.argmax(logits, axis=-1)
			output_pred.append(result)
			output_prob.append(prob)

	return np.concatenate(output_pred).tolist(), np.concatenate(output_prob, axis=0).tolist()

def num_to_label(label):
	"""
	숫자로 되어 있던 class를 원본 문자열 라벨로 변환 합니다.
	"""
	origin_label = []
	with open('dict_num_to_label.pkl', 'rb') as f:
		dict_num_to_label = pickle.load(f)
	for v in label:
		origin_label.append(dict_num_to_label[v])
	return origin_label

def label_to_num(label):
    num_label = []
    with open('dict_label_to_num.pkl', 'rb') as f:
        dict_label_to_num = pickle.load(f)
    for v in label:
        num_label.append(dict_label_to_num[v])
    
    return num_label

In [17]:
set_seed(42)

In [18]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)
MODEL_NAME = 'klue/roberta-large'
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model_config =  AutoConfig.from_pretrained(MODEL_NAME)
model = R_BigBird(model_config, 0.1)
model.load_state_dict(torch.load('/opt/ml/code/best_model/1_best_model/pytorch_model.bin'))
model.to(device)

dataset = load_data('../dataset/train/train.csv')
tokenized_train = tokenized_dataset(dataset, tokenizer)
tokenized_train = make_entity_mask(tokenized_train)
train_label = label_to_num(dataset['label'].values)
RE_dataset_test = RE_Dataset(tokenized_train, train_label)

cuda:0


Some weights of the model checkpoint at klue/roberta-large were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.decoder.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at klue/roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it f

In [19]:
output_pred, output_prob = inference(model, RE_dataset_test, device)
original_label = num_to_label(output_pred)

100%|██████████| 1014/1014 [10:27<00:00,  1.62it/s]


## 어떤걸 못맞출까?

In [20]:
dataset['pred'] = original_label

In [21]:
dataset[dataset['pred'] != dataset['label']]['pred'].value_counts()
# 틀린 label의 갯수

per:alternate_names                    1719
org:alternate_names                     682
per:employee_of                         583
per:origin                              565
org:place_of_headquarters               426
per:title                               389
org:top_members/employees               324
org:members                             310
no_relation                             279
org:member_of                           239
per:place_of_residence                   88
per:place_of_birth                       72
per:colleagues                           68
org:product                              64
org:founded                              60
per:spouse                               57
per:product                              48
per:parents                              42
per:religion                             41
per:date_of_birth                        38
org:political/religious_affiliation      37
per:other_family                         21
per:children                    

In [22]:
false_df = dataset[dataset['label'] != dataset['pred']]

In [23]:
false_count = pd.DataFrame(false_df.groupby(['label','pred'])['pred'].count())
false_count = false_count.stack().reset_index().drop('level_2',axis=1).rename(columns={0:'count'})
false_count.head()

,label,pred,count
0,no_relation,org:alternate_names,477
1,no_relation,org:dissolved,14
2,no_relation,org:founded,56
3,no_relation,org:founded_by,5
4,no_relation,org:member_of,206


In [24]:
false_count[false_count['label'] == 'no_relation'].sort_values('count', ascending=False)
# 정답이 no_realtion인데 잘못 예측한 것의 갯수

,label,pred,count
11,no_relation,per:alternate_names,875
16,no_relation,per:employee_of,527
0,no_relation,org:alternate_names,477
28,no_relation,per:title,372
7,no_relation,org:place_of_headquarters,310
10,no_relation,org:top_members/employees,293
17,no_relation,per:origin,253
5,no_relation,org:members,244
4,no_relation,org:member_of,206
13,no_relation,per:colleagues,67


* 정답이 no_realtion인데 어떻게 잘못예측하는지 살펴봤다.
* 이제부턴, 가설 '같은 sentence 인데 label이 no_relation 그리고 다른 label로 설정이 되어있어서 모델이 헷갈리는 것이다.'를 검증해보자.
* 검증방법: 중복 sentence 중 label이 no_relation을 포함하고 최소 하나 이상의 no_relation이 아닌 label이 포함된 row들의 분포를 살펴보자.

In [25]:
dup_df = dataset[dataset.duplicated('sentence', keep=False)].sort_values('sentence')
dup_df

,id,sentence,subject_entity,object_entity,label,source,pred
918,919,"""2006년 지방선거에 참패하고 민주당과 그 전신인 열린 우리당은 노 씨의 국정 운...","{'word': '열린 우리당', 'start_idx': 29, 'end_idx':...","{'word': '노무현', 'start_idx': 127, 'end_idx': 1...",no_relation,wikipedia,no_relation
14724,14736,"""2006년 지방선거에 참패하고 민주당과 그 전신인 열린 우리당은 노 씨의 국정 운...","{'word': '열린 우리당', 'start_idx': 29, 'end_idx':...","{'word': '정세균', 'start_idx': 108, 'end_idx': 1...",no_relation,wikipedia,org:top_members/employees
3586,3588,"""소이현, 인교진이 하는 광고는 괜찮고"", ""차 광고할 때 여자가 남자한테 하던 소...","{'word': '인교진', 'start_idx': 6, 'end_idx': 8, ...","{'word': '소이현', 'start_idx': 52, 'end_idx': 54...",no_relation,wikitree,per:alternate_names
28018,28051,"""소이현, 인교진이 하는 광고는 괜찮고"", ""차 광고할 때 여자가 남자한테 하던 소...","{'word': '소이현', 'start_idx': 52, 'end_idx': 54...","{'word': '인교진', 'start_idx': 6, 'end_idx': 8, ...",no_relation,wikitree,per:alternate_names
19411,19427,"""탕약망""이란 예수회 선교사 천문학자 아담 샬의 중국 이름이다.","{'word': '탕약망', 'start_idx': 1, 'end_idx': 3, ...","{'word': '예수회', 'start_idx': 8, 'end_idx': 10,...",per:employee_of,wikipedia,per:employee_of
...,...,...,...,...,...,...,...
1620,1621,힐러리 클린턴 이메일 논쟁은 2016년 미국 대통령 선거에서 미국 공화당 도널드 트...,"{'word': '힐러리 클린턴', 'start_idx': 60, 'end_idx'...","{'word': '민주당', 'start_idx': 56, 'end_idx': 58...",per:employee_of,wikipedia,per:employee_of
2538,2539,힐러리 클린턴은 2016년 7월 26일 필라델피아에서 열린 민주당 전당대회에서 공식...,"{'word': '힐러리 클린턴', 'start_idx': 0, 'end_idx':...","{'word': '민주당', 'start_idx': 50, 'end_idx': 52...",per:employee_of,wikipedia,per:employee_of
20884,20903,힐러리 클린턴은 2016년 7월 26일 필라델피아에서 열린 민주당 전당대회에서 공식...,"{'word': '민주당', 'start_idx': 50, 'end_idx': 52...","{'word': '2016년', 'start_idx': 9, 'end_idx': 1...",no_relation,wikipedia,no_relation
24268,24292,힙합 그룹 에픽하이의 리더인 타블로는 멤버들의 군 입대와 학력 논란으로 인한 공백을...,"{'word': '타블로', 'start_idx': 16, 'end_idx': 18...","{'word': '에픽하이', 'start_idx': 6, 'end_idx': 9,...",per:employee_of,wikipedia,per:employee_of


In [26]:
def check(row):
    return row['label'].value_counts().index
        
df = pd.DataFrame(dup_df.groupby(['sentence']).apply(check)).stack().reset_index()

In [27]:
def check_no(row):
    row = list(row)
    if 'no_relation' in row and len(row)>1:
        row.remove('no_relation')
        return row
    else: return np.nan

In [28]:
df[0].map(check_no).dropna().explode().value_counts()
# 위에 분포와 비교해보면 확실히 비슷한 것을 확인할 수 있음.

per:employee_of                        172
org:top_members/employees              106
org:member_of                           99
per:title                               84
per:colleagues                          54
per:date_of_birth                       38
org:alternate_names                     35
org:place_of_headquarters               30
per:origin                              29
per:alternate_names                     28
per:parents                             27
org:members                             26
per:other_family                        21
per:spouse                              17
per:children                            16
per:place_of_residence                  12
per:product                             11
org:product                             10
org:founded_by                           8
per:date_of_death                        8
org:founded                              7
org:political/religious_affiliation      6
per:siblings                             6
per:place_o

In [38]:
dataset = load_data('../dataset/train/train.csv')
dataset['duplicated'] = dataset.duplicated('sentence', keep=False)
dataset = dataset.sort_values(['sentence','label'], ascending=False)

In [39]:
prev_sen, flag, tt = None, False, []

for row in dataset.itertuples():
    if row.duplicated == True and row.label != 'no_relation':
       flag = True 
       tt.append(False)
       prev_sen = row.sentence
       continue

    if row.sentence == prev_sen and flag == True:
        if row.label == 'no_relation':
            tt.append(True)
            prev_sen = row.sentence
            continue
            
    flag = False
    tt.append(False)
    prev_sen = row.sentence

dataset['condition'] = tt

In [40]:
dataset[dataset['condition'] == True]['label'].value_counts()

no_relation    865
Name: label, dtype: int64

## train, test에도 중복되는게 있을까?

In [27]:
train = load_data('../dataset/train/train.csv')
test = load_data('../dataset/test/test_data.csv')

In [28]:
train['type'] = 'train'
test['type'] = 'test'

In [39]:
df = pd.concat([train,test])
dup_df = df[df.duplicated('sentence',keep=False)]

In [41]:
dup_df[dup_df['type'] == 'test']

,id,sentence,subject_entity,object_entity,label,source,type
6,6,"한국당 전희경 대변인은 이날 정 총리 후보자 지명 직후 논평을 내고 ""의회를 시녀화...","{'word': '전희경', 'start_idx': 4, 'end_idx': 6, ...","{'word': '한국당', 'start_idx': 0, 'end_idx': 2, ...",100,wikitree,test
7,7,문재인 대통령 부인 김정숙 여사는 22일부터 1박2일 일정으로 광주를 방문해 경기를...,"{'word': '문재인', 'start_idx': 0, 'end_idx': 2, ...","{'word': '김정숙', 'start_idx': 11, 'end_idx': 13...",100,wikitree,test
31,31,서방의 군사 전문가들은 비하치 포위망 주변 크로아티아 영토에 있는 세르비아군의 지대...,"{'word': '세르비아군', 'start_idx': 37, 'end_idx': ...","{'word': '크로아티아', 'start_idx': 24, 'end_idx': ...",100,wikipedia,test
33,33,FC 바르셀로나와 아틀레티코 마드리드의 유명한 골키퍼였던 미겔 레이나의 아들인 페페...,"{'word': '프리메라리가', 'start_idx': 86, 'end_idx':...","{'word': '아틀레티코 마드리드', 'start_idx': 10, 'end_i...",100,wikipedia,test
148,148,이번 강연은 김초혜 시인의 시 세계를 알 수 있는 특별강연과 독자의 이야기를 들어볼...,"{'word': '조정래', 'start_idx': 85, 'end_idx': 87...","{'word': '작가', 'start_idx': 89, 'end_idx': 90,...",100,wikitree,test
...,...,...,...,...,...,...,...
7353,7353,더군다나 조정은 건륭제의 사돈이자 이미 30여년간 조정을 장악하던 수석군기대신 겸 ...,"{'word': '건륭제', 'start_idx': 9, 'end_idx': 11,...","{'word': '가경제', 'start_idx': 67, 'end_idx': 69...",100,wikipedia,test
7365,7365,정통 뉴올리언즈 핫 재즈 스타일 음악을 추구하는 스웨덴의 남성 6인조 밴드 젠틀맨 ...,"{'word': '국립현대무용단', 'start_idx': 144, 'end_idx...","{'word': '6인조', 'start_idx': 35, 'end_idx': 37...",100,wikitree,test
7476,7476,"야마지 모토하루(山地元治, 1841년 9월 10일 - 1897년 10월 3일)는 도...","{'word': '야마지 모토하루', 'start_idx': 0, 'end_idx'...","{'word': '1841년 9월 10일', 'start_idx': 15, 'end...",100,wikipedia,test
7502,7502,"오비맥주 고동우 대표, 전국모범운전자연합회 윤석범 회장, 대한민국 1등 주차앱 모두...","{'word': '전국모범운전자연합회', 'start_idx': 13, 'end_i...","{'word': '대표', 'start_idx': 56, 'end_idx': 57,...",100,wikitree,test
